In [2]:
import os
import shutil
import re
import subprocess
import math

from IPython.core.debugger import set_trace

In [47]:
# Given a complete filepath to a video, returns an integer of the number of frames in the video.
def video_data(filename):
    # Run FFmpeg to get information on the given video
    output = subprocess.run(['ffmpeg', '-i', filename, '-map', '0:v:0', '-c', 'copy', '-f', 'null', '-'], capture_output = True, text = True)
    # FFmpeg return a CompletedProcess data type. In order to process the information, change it to a string data type.
    output = str(output)
    # Use regular expression matching to extract video data
    try:
        frame_text = re.findall('frame= *\d+', output)[0] # Regular expression: "frame=" + (zero or more spaces or tabs) + (one or more digits)
    except:
        set_trace()
    resolution_text = re.findall(', \d+x\d+', output)[0]
    # Get the number that comes after 'frame ='
    frame_num = frame_text.split(' ')[-1]
    width = int(''.join(char for char in resolution_text.split('x')[0] if char.isdigit()))
    height = int(''.join(char for char in resolution_text.split('x')[1] if char.isdigit()))
    return (frame_num, width, height)

In [ ]:
video_data(r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs\2019 AERIAL _ Stock Footage\2019 AERIAL _ Stock Footage_half_size.mp4')

In [7]:
# Change the working directory to the folder where the downloaded, full-length videos are stored, each in their own individual folder
os.chdir(r'Raw Data\Raw, Half-Size, and PNGs')

In [40]:
os.chdir('2019 AERIAL _ Stock Footage')

In [53]:
len('C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs\Bourbon Sweet Potato Casserole with Sweet ‘n’ Savory Bacon Pecans')

SyntaxError: EOL while scanning string literal (<ipython-input-53-ae0e8ab1a6a0>, line 1)

In [37]:
for folder in os.listdir(os.getcwd()):
    os.chdir(folder)
    filename = list(filter(lambda x: x.endswith('_half_size.mp4'), os.listdir(os.getcwd())))[0] # Filename of the half-size video
    num_frames, width, height = video_data(os.getcwd() + '\\' + filename)

    new_width = width - (width % 256)
    new_height = height - (height % 256)
    cropped_filename = filename[:-4] + '_cropped.mp4'
    crop_command = 'crop=' + str(new_width) + ':' + str(new_height) + ':' \
    + str(math.floor((width % 265) / 2)) + ':' + str(math.floor((height % 265) / 2))
    subprocess.run(['ffmpeg', '-y', '-i', filename, '-filter:v', crop_command, cropped_filename])
    for x_pixel_index in list(range(0, new_width, 256)):
        for y_pixel_index in list(range(0, new_height, 256)):
            crop_command = 'crop=' + ':'.join(['256', '256', str(x_pixel_index), str(y_pixel_index)]) + ''
            patch_filename = cropped_filename[:-4] + str(int(x_pixel_index / 256)) + str(int(y_pixel_index / 256)) + '.mp4'
            subprocess.run(['ffmpeg', '-y', '-i', cropped_filename, '-filter:v', crop_command, patch_filename])
            if os.path.exists(patch_filename[:-4]):
                shutil.rmtree(patch_filename[:-4])
            os.mkdir(patch_filename[:-4])
            shutil.move(patch_filename, patch_filename[:-4] + '\\' + patch_filename)

    os.chdir('..')   

In [39]:
os.g

'C:\\Users\\John\\PythonVenvs\\VideoSuperResolution\\Scripts\\Raw Data\\Raw, Half-Size, and PNGs'

In [51]:
# For each file in the directory, split the full-length videos into individual .png images
# os.getcwd() returns a string of the current directory (the one set up above). os.listdir() then list every file in that directory.
for folder in [x for x in os.listdir(os.getcwd()) if not x.endswith('.bat')][4:]:
    os.chdir(folder)
    for patch_folder in (x for x in os.listdir(os.getcwd()) if not x.endswith('.mp4')):
        os.chdir(patch_folder)
        filename = os.listdir(os.getcwd())[0] # Filename of the patch of video
        num_frames, a, b = video_data(os.getcwd() + '\\' + filename)
        subprocess.run(['ffmpeg', '-i', filename, '-vf', 'select=\'between(n\\,0\\,' + num_frames + ')\'', '-vsync', '0', '-start_number', '0', 'frame%d.png'], check=True)
        os.chdir('..') # Back up to the directory with the list of patch folders
    
    os.chdir('..') # Back up to the main video directory
    

> <ipython-input-47-56704b2f6e4f>(12)video_data()
     10     except:
     11         set_trace()
---> 12     resolution_text = re.findall(', \d+x\d+', output)[0]
     13     # Get the number that comes after 'frame ='
     14     frame_num = frame_text.split(' ')[-1]



ipdb>  output


"CompletedProcess(args=['ffmpeg', '-i', 'C:\\\\Users\\\\John\\\\PythonVenvs\\\\VideoSuperResolution\\\\Scripts\\\\Raw Data\\\\Raw, Half-Size, and PNGs\\\\Bourbon Sweet Potato Casserole with Sweet ‘n’ Savory Bacon Pecans\\\\Bourbon Sweet Potato Casserole with Sweet ‘n’ Savory Bacon Pecans_half_size_cropped00\\\\Bourbon Sweet Potato Casserole with Sweet ‘n’ Savory Bacon Pecans_half_size_cropped00.mp4', '-map', '0:v:0', '-c', 'copy', '-f', 'null', '-'], returncode=1, stdout='', stderr='ffmpeg version git-2020-08-31-4a11a6f Copyright (c) 2000-2020 the FFmpeg developers\\n  built with gcc 10.2.1 (GCC) 20200805\\n  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheo

ipdb>  q


BdbQuit: 

In [32]:
# Halve the raw video sizes to get the HR frames 
os.chdir(r'Raw Data\Just Raw Videos')

for folder in os.listdir(os.getcwd()):
    os.chdir(folder)
    filename = os.listdir(os.getcwd())[0] # Filename of the only file in the folder. Each folder should have exactly one video file
    # ffmpeg = subprocess.run(['ffmpeg', '-i', "\"" + filename + "\"", '-vf', '"scale=iw/2:ih/2"', "\"" + filename[:-4] + '_half_size.mp4' + "\""])
    ffmpeg = subprocess.run(['ffmpeg', '-y', '-i', filename, '-vf', 'scale=iw/2:ih/2', filename[:-4] + '_half_size.mp4'])
    os.chdir('..') # Back up to the directory with the list of video folders


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Raw Data\\Just Raw Videos'

In [50]:
os.chdir(r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs')
print(os.getcwd())

C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs
